In [ ]:
# | include: false
# | default_exp utils

In [ ]:
# | export

import _ast
import ast
import os
import subprocess
import sys
from pathlib import Path
from typing import Tuple

from execnb.nbio import read_nb
from fastcore.basics import AttrDict
from nbdev.config import get_config
from nbqa.find_root import find_project_root

In [ ]:
%load_ext autoreload
%autoreload 2

# Test Setup

In [ ]:
test_nb_path = Path("test", "test_export.ipynb")

# Subprocess

## `run_shell_cmd`

In [ ]:
# | export


def run_shell_cmd(script: str):
    pipe = subprocess.Popen(
        "%s" % script, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True
    )
    output = pipe.communicate()[0]
    return pipe, output.decode("utf-8").strip()

In [ ]:
cmd_result = run_shell_cmd("pwd")
assert cmd_result[0].returncode == 0
assert cmd_result[1].find("sciflow") > 0

## `run_py_module`

In [ ]:
# | export


def run_py_module(command, args=None, env=None):
    if args is None:
        cmd = [sys.executable, "-m", command]
    else:
        cmd = [sys.executable, "-m", command, *(str(i).strip() for i in args)]
    output = subprocess.run(
        cmd,
        stderr=subprocess.PIPE,
        stdout=subprocess.PIPE,
        env=env,
        universal_newlines=True
    )

    output_code = output.returncode
    output.stdout
    err = output.stderr

    if output_code != 0:
        raise EnvironmentError(err)
    return output_code

In [ ]:
assert run_py_module("pigar") == 0

# Code-gen

# `indent_multiline`

In [ ]:
# | export


def indent_multiline(multiline_text, indent=1):
    lines = multiline_text.strip().split("\n")
    spaces = "".join(["    " for _ in range(indent)])
    for i in range(len(lines)):
        prefix = spaces if i > 0 else spaces + '"""'
        lines[i] = prefix + lines[i]
    return "\n".join(lines) + '"""'

In [ ]:
text = """
Some text
:param param: text
"""
assert '    """Some text\n    :param param: text"""' == indent_multiline(text)

# Text

# `titleize`

In [ ]:
# | export


def titleize(name):
    return name.title().replace("_", "")

In [ ]:
assert titleize("snake_case") == "SnakeCase"

# Collections

# `chunks`

In [ ]:
# | export


def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

In [ ]:
assert list(chunks([1,2,3,4,5,6], 3)) == [[1,2, 3], [4, 5, 6]]
assert list(chunks([1,2,3,4,5], 3)) == [[1,2, 3], [4, 5]]

# Paths

# `lib_path`

In [ ]:
# | export


def lib_path(*lib_relative_path):
    lib_root_path = find_project_root(srcs=(str(Path(".").resolve()),))
    return Path(os.path.join(lib_root_path, *lib_relative_path))

In [ ]:
assert str(lib_path("nbs")).endswith("sciflow/nbs")
assert Path("test/test_multistep.ipynb").resolve() == lib_path(
    "nbs", "test", "test_multistep.ipynb"
)

# Notebook

# `find_default_export`

In [ ]:
# | export


def find_default_export(cells):
    default_export = None
    cell_lines = [c["source"].split("\n") for c in cells]
    for cell in cell_lines:
        for line in cell:
            trimmed_line = line.strip().replace(" ", "")
            if trimmed_line.startswith("#|default_exp"):
                default_export = trimmed_line.split("#|default_exp")[1].strip()
    return default_export

In [ ]:
assert "test.test_export" == find_default_export(read_nb(test_nb_path)["cells"])

# `load_nb`

Loads notebook and provides companion module path

In [ ]:
# | export


def load_nb(nb_path: Path) -> Tuple[AttrDict, Path]:
    nb = read_nb(nb_path)
    default_export = find_default_export(nb["cells"])
    if default_export is None:
        raise ValueError(f"{nb_path.name} does not contain an associated nbdev module")

    module_name = default_export.replace(".", "/")
    module_path = Path(get_config().path("lib_path"), f"{module_name}.py")
    return nb, module_path

In [ ]:
from nbdev.export import nb_export

nb_export("test/test_multistep.ipynb")

In [ ]:
nb, module_path = load_nb(Path("test/test_multistep.ipynb"))
assert type(nb) == AttrDict
module_path.exists()

True

In [ ]:
check = False
try:
    nb, module_path = load_nb(Path(Path(".").resolve(), "index.ipynb"))
except ValueError:
    check = True
assert check

# `prepare_env`

In [ ]:
# | export


def prepare_env(env_file_path: str = None):
    if env_file_path is None:
        env_file_path = os.path.expanduser("~/.sciflow/env")
    if not os.path.exists(env_file_path):
        raise EnvironmentError(
            f"You need to create a Sciflow environment vars file at: {env_file_path}"
        )
    with open(env_file_path, "r") as env_file:
        for line in env_file.readlines():
            key, value = line.strip().split("=", 1)
            os.environ[key.replace("export ", "")] = value
        python_path = [
            p for p in os.environ["PYTHONPATH"].split(":") if p != "$PYTHONPATH"
        ]
        sys.path.extend(python_path)

# Flows

# `load_nb_module`

In [ ]:
# | export


def load_nb_module(nb_path):
    nb, module_path = load_nb(nb_path)
    with open(module_path, "r") as module_file:
        lines = module_file.readlines()
    module_code = "\n".join(lines)
    return nb, module_code

In [ ]:
nb, module_code = load_nb_module(Path("test/test_multistep.ipynb"))

In [ ]:
assert type(nb) == AttrDict
assert type(ast.parse(module_code)) == _ast.Module

In [ ]:
# | export


def get_module_name(nb_path):
    nb = read_nb(nb_path)
    module_name = find_default_export(nb["cells"])
    return module_name

In [ ]:
assert "test.test_export" == get_module_name(test_nb_path)

# `get_flow_path`

In [ ]:
# | export


def get_flow_path(nb_path, config=None, flow_provider="metaflow"):
    module_name = get_module_name(nb_path)
    if module_name is None:
        return None
    if config is None:
        config = get_config()
    flows_dir = Path(config.path("flows_path"), flow_provider)
    if not flows_dir.exists():
        flows_dir.mkdir(parents=True)

    return Path(flows_dir, f"{module_name.split('.')[-1]}.py")

In [ ]:
assert Path("test", "flows", "metaflow", "test_export.py").resolve() == get_flow_path(test_nb_path)